In [1]:
(
    1 + 0
)  # Utility cell. Sometimes "Jupyter" skipps the first cell when running all cells. Especially after a restart.

1

In [2]:
(
    1 + 1
)  # Utility cell. Sometimes "Jupyter" skipps the first cell when running all cells. Especially after a restart.

2

In [3]:
print(f"N{1+2} utility cell running...")

N3 utility cell running...


In [4]:
# Resolve the path to the 'src' folder with helper functions
import config_notebooks

PYTHONPATH: C:\Users\Admin\Documents\Univer\MASTER THESIS IDEAS\Flavio Pinehro\11_MasterProject\src


In [5]:
from n04_import import *

os.getcwd():
c:\Users\Admin\Documents\Univer\MASTER THESIS IDEAS\Flavio Pinehro\11_MasterProject\notebooks
input_file_location:
..\data\03_preprocessed
output_file_location:
..\data\04_mor_implemented


In [6]:
# # Resolve the path to the 'src' folder with helper functions
# import config_notebooks
# from importlib import reload

# # Import the module and reload it
# import n04_import_reload

# reload(n04_import_reload)

# # Reload updated functions
# from n04_import_reload import *

Restoring reliable gdp country codes

In [7]:
initial_input_location = input_file_location

input_file_location = os.path.join("..", "data", "02_ingested")
dumped_data = restore_cache(input_file_location=input_file_location)

# This data was staged at n01_Ingested_gdp.ipynb
valid_gdp_locations = dumped_data["valid_gdp_locations"]

# This data was staged at n03_Processed_gdp.ipynb
reliable_gdp_codes = dumped_data["reliable_gdp_codes"]

input_file_location = initial_input_location

data restored:['valid_gdp_country_codes', 'valid_gdp_country_ids', 'valid_gdp_locations', 'export_years', 'unreliable_locations', 'rolling_exports_years', 'reliable_gdp_codes']


In [8]:
print(reliable_gdp_codes)

{'NPL', 'HRV', 'THA', 'RUS', 'ZAF', 'BGR', 'DOM', 'JPN', 'CIV', 'GAB', 'AUS', 'GTM', 'ALB', 'UZB', 'MOZ', 'ITA', 'RWA', 'NOR', 'CZE', 'TUR', 'OMN', 'LBR', 'FRA', 'MYS', 'MEX', 'DZA', 'COG', 'NAM', 'KAZ', 'BEL', 'GHA', 'CMR', 'MDA', 'AZE', 'TJK', 'BOL', 'LSO', 'CRI', 'SOM', 'BEN', 'UKR', 'NGA', 'FIN', 'QAT', 'IRL', 'HUN', 'LBN', 'ISR', 'MRT', 'MDG', 'POL', 'MLI', 'COD', 'ESP', 'KOR', 'PHL', 'SLE', 'EGY', 'LAO', 'AUT', 'TUN', 'BFA', 'HKG', 'MMR', 'LTU', 'SRB', 'MUS', 'ARE', 'SAU', 'BRA', 'PAK', 'GBR', 'BIH', 'AFG', 'NZL', 'ZMB', 'DNK', 'GRC', 'SYR', 'AGO', 'CAN', 'GEO', 'TCD', 'IND', 'SVN', 'NER', 'USA', 'CHN', 'HND', 'SGP', 'TTO', 'KWT', 'PRY', 'MKD', 'PER', 'ECU', 'SEN', 'PAN', 'ETH', 'PSE', 'BDI', 'GMB', 'YEM', 'SDN', 'LBY', 'ARM', 'IRQ', 'COL', 'ARG', 'BGD', 'IDN', 'BWA', 'CHL', 'CAF', 'PRT', 'KEN', 'EST', 'LKA', 'SLV', 'SWE', 'URY', 'HTI', 'JOR', 'TKM', 'IRN', 'ZWE', 'TGO', 'MWI', 'ROU', 'GIN', 'BLR', 'PNG', 'SVK', 'NIC', 'KGZ', 'MAR', 'DEU', 'CUB', 'CHE', 'VNM', 'MNG', 'LVA', 'UGA'

### Will be calculating given order of reflections

In [9]:
order_of_reflections = 18

Rading location-activities matrix X_cp, i.e. volume of activity p in location c

In [10]:
input_file_name = "raw_X_cp.parquet"
x_cp = read_hs92_parquet_data(input_file_location, input_file_name, f_convert_dtype=False)

In [11]:
df_stats(x_cp, "x_cp")

['x_cp', <class 'pandas.core.frame.DataFrame'>]
shape:
(1408240, 4)
data types:
country_id                int16
product_id                int16
year                      int16
average_export_value    float64
dtype: object
columns:
['country_id', 'product_id', 'year', 'average_export_value']
        country_id   product_id         year  average_export_value
count 1,408,240.00 1,408,240.00 1,408,240.00          1,408,240.00
mean        443.10     1,270.10     2,015.50         92,694,429.21
std         251.82       358.53         2.29      1,431,323,301.33
min           8.00       650.00     2,012.00                  0.00
25%         222.00       961.00     2,013.75                  0.00
50%         428.00     1,270.50     2,015.50            132,606.33
75%         682.00     1,585.00     2,017.25          4,608,486.33
max         894.00     1,892.00     2,019.00    359,983,930,717.33
Total memory usage: 19715492 bytes


In [12]:
x_cp.head()

,country_id,product_id,year,average_export_value
0,8,650,2012,0.00
1,8,650,2013,"179,937.67"
2,8,650,2014,"179,937.67"
3,8,650,2015,"179,937.67"
4,8,650,2016,0.00


Removing countries which are not in reliable_gdp_codes. </br></br>To do so we take valid_gdp_locations dataframe with "country_id" and "country_code" columns, and which accounts only for countries with "significant enough" gdp per capita value by the year of 2010. We will extract those "country_id" which "country_code" is in reliable_gdp_codes variable, which was identified in n03_Processed_gdp file, and which holds country codes only for those countries, which have consistent gdp per capita data across all export_years.

In [13]:
valid_gdp_locations.sample(5)

,country_id,country_code
57,348,HUN
83,498,MDA
62,368,IRQ
123,706,SOM
56,332,HTI


In [14]:
valid_gdp_locations.dtypes

country_id       int16
country_code    object
dtype: object

In [15]:
reliable_gdp_ids = set(
    valid_gdp_locations[valid_gdp_locations["country_code"].isin(reliable_gdp_codes)][
        "country_id"
    ].unique()
)

In [16]:
type(reliable_gdp_ids), len(reliable_gdp_ids)

(set, 147)

In [17]:
x_cp = x_cp[x_cp["country_id"].isin(reliable_gdp_ids)].reset_index(drop=True)

In [18]:
df_stats(x_cp, "x_cp")

['x_cp', <class 'pandas.core.frame.DataFrame'>]
shape:
(1398528, 4)
data types:
country_id                int16
product_id                int16
year                      int16
average_export_value    float64
dtype: object
columns:
['country_id', 'product_id', 'year', 'average_export_value']
        country_id   product_id         year  average_export_value
count 1,398,528.00 1,398,528.00 1,398,528.00          1,398,528.00
mean        440.19     1,270.10     2,015.50         93,049,051.28
std         250.26       358.53         2.29      1,433,348,437.14
min           8.00       650.00     2,012.00                  0.00
25%         221.00       961.00     2,013.75                  0.00
50%         427.00     1,270.50     2,015.50            136,205.50
75%         655.00     1,585.00     2,017.25          4,705,561.92
max         894.00     1,892.00     2,019.00    359,983,930,717.33
Total memory usage: 19579524 bytes


In [19]:
# Save clean X_cp matrix to a .parquet file
output_file_name = "X_cp.parquet"
save_hs92_parquet_data(
    data=x_cp,
    file_location=input_file_location,
    file_name=output_file_name,
)

country_id column initial type:
int16
country_id column final type:
int16
product_id column initial type:
int16
product_id column final type:
int16
year column initial type:
int16
year column final type:
int16


## Implementing Method of Reflections

Calculating RCA metric</br>
Revealed Comparative Advantage (RCA) metric in this scenario is used to normalize location-activities $X_{cp}$ matrix

In [20]:
categorical_columns = ["country_id", "product_id", "year"]
x_cp = switch_to_dtype(x_cp, default_dtype="category", categorical_columns=categorical_columns)

country_id column initial type:
int16
country_id column final type:
category
product_id column initial type:
int16
product_id column final type:
category
year column initial type:
int16
year column final type:
category


In [21]:
x_cp = x_cp.sort_values(by=categorical_columns)

In [22]:
# Will consider product_rolling_exports as an input x_cp matrix, and calculate RCA
x_cp = calculate_RCA(x_cp)

In [23]:
RCA = x_cp[["country_id", "product_id", "year", "RCA"]]

In [24]:
df_stats(RCA, "RCA")

['RCA', <class 'pandas.core.frame.DataFrame'>]
shape:
(1398528, 4)
data types:
country_id    category
product_id    category
year          category
RCA            float64
dtype: object
columns:
['country_id', 'product_id', 'year', 'RCA']
               RCA
count 1,395,360.00
mean          1.45
std          28.25
min           0.00
25%           0.00
50%       0.046518
75%       0.415679
max       7,624.32
Total memory usage: 18207340 bytes


Staging RCA matrix

In [25]:
# Save the product_exports_total average Data to a raw_product_exports_total.parquet file
output_file_name = "RCA.parquet"
save_hs92_parquet_data(
    data=RCA, file_location=output_file_location, file_name=output_file_name
)

country_id column initial type:
category
country_id column final type:
int16
product_id column initial type:
category
product_id column final type:
int16
year column initial type:
category
year column final type:
int16


In [26]:
del x_cp, RCA
gc.collect()

16

Reading RCA matrix

In [27]:
# # Resolve the path to the 'src' folder with helper functions
# import config_notebooks
# from importlib import reload

# # Import the module and reload it
# import n04_import_reload

# reload(n04_import_reload)

# # Reload updated functions
# from n04_import_reload import *

In [28]:
input_file_name = "RCA.parquet"
RCA = read_hs92_parquet_data(output_file_location, input_file_name)

country_id column initial type:
int16
country_id column final type:
category
product_id column initial type:
int16
product_id column final type:
category
year column initial type:
int16
year column final type:
category


In [29]:
df_stats(RCA, "RCA")

['RCA', <class 'pandas.core.frame.DataFrame'>]
shape:
(1398528, 4)
data types:
country_id    category
product_id    category
year          category
RCA            float64
dtype: object
columns:
['country_id', 'product_id', 'year', 'RCA']
               RCA
count 1,395,360.00
mean          1.45
std          28.25
min           0.00
25%           0.00
50%       0.046518
75%       0.415679
max       7,624.32
Total memory usage: 18207340 bytes


In [30]:
RCA.head()

,country_id,product_id,year,RCA
0,8,650,2012,0.00
1,8,651,2012,0.00
2,8,652,2012,0.00
3,8,653,2012,0.00
4,8,654,2012,0.00


In [31]:
RCA["year"].unique()

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
Categories (8, int16): [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [32]:
# Calculating unique values for country_id and product_id
RCA[["country_id", "product_id"]].apply(lambda col: col.unique().size)

country_id     144
product_id    1214
dtype: int64

In [33]:
# Exploring the exports_summary data
RCA_unstacked = RCA[RCA["year"] == 2012].pivot(
    index=["country_id"], columns="product_id", values=["RCA"]
)
# RCA_unstacked.head(10)
RCA_unstacked.sample(10)

RCA                                                        \
product_id      650      651      652      653      654      655      656   
country_id                                                                  
818        0.298866     0.00 0.003466 0.233199 0.024720     6.45 0.019332   
178            0.00 0.005465     0.00     0.00     0.00     2.45     0.00   
795            0.00     0.00     0.00     0.00     0.00     0.00     0.00   
72         0.011438     1.95     0.00 0.044249 0.018191 0.967189     5.08   
324            0.00     0.00     0.00     0.00     0.00     9.88     0.00   
480        0.440319     0.00     0.00     0.00     3.29   174.96 0.003026   
704        0.000643 0.000891     0.00 0.000339     0.00 0.588645 0.000301   
710        0.463576 0.237756 0.021492 0.234362 0.603307     2.34 0.217118   
68             0.00     0.00     0.00     0.00     0.00     0.00 0.052712   
188        0.078392 0.148648 0.051964 0.000879 0.744367 0.687757 0.484120   

                                       ...                             \
product_id      657      658      659  ...     1883     1884     1885   
country_id                             ...                              
818        0.005560 0.001201 0.024497  ... 0.060179 0.476423 0.039921   
178        0.003495 0.005424     0.00  ...     0.00 0.014951 0.000791   
795            0.00 0.000187     0.00  ... 0.006417     0.00 0.001951   
72             6.61 0.016396 0.004393  ...     0.00     0.00 0.005351   
324            0.00     0.00 0.039736  ...     0.00     0.00 0.006860   
480        0.042508 0.008036 0.052639  ... 0.061611     5.10 0.077110   
704        0.010973 0.305871 0.002816  ... 0.070886     1.56 0.022893   
710        0.123395 0.096997 0.127329  ... 0.072308 0.336497 0.150347   
68         0.041650     0.00     0.00  ...     0.00 0.015405 0.005382   
188            1.51 0.033267 0.001647  ... 0.006297 0.003859 0.036047   

                                                                           
product_id     1886     1887     1888     1889     1890     1891     1892  
country_id                                                                 
818        0.029038 0.163254 0.130502     1.45     4.33 0.055970     1.38  
178            0.00 0.031217 0.017688 0.287208     3.12 0.014251 0.436535  
795            0.00 0.001893     0.00 0.070103 0.497728 0.105357 0.376359  
72         0.128204 0.026273 0.053069 0.267562 0.127978 0.057188     1.21  
324            0.00 0.074629 0.291265 0.229064 0.564551 0.103099     0.00  
480        0.034105 0.119330     0.00 0.164474 0.031687 0.068256     0.00  
704            0.00 0.012293 0.003998 0.022262 0.043144 0.146671     1.72  
710        0.255758 0.182300 0.370620     2.97 0.036380 0.069285     8.31  
68         0.016812 0.002165 0.026683 0.036026 0.058380 0.009213 0.172437  
188        0.145014 0.035390     0.00 0.011434 0.011800 0.256390    30.42  

[10 rows x 1214 columns]

In [34]:
del RCA_unstacked
gc.collect()

0

In [35]:
# assert False, "End of code"

Defining the binary specialization matrix $M_{cp}$
</br>$M$ helps remove excess variation by focusing only on significant presences ($M_{cp} = 1$) and absences ($M_{cp} = 0$)

In [36]:
# Converting RCA into binary matrix M_cp
M_cp = create_M_matrix(RCA)

In [37]:
df_stats(M_cp, "M_cp")

['M_cp', <class 'pandas.core.frame.DataFrame'>]
shape:
(1398528, 4)
data types:
country_id    category
product_id    category
year          category
M                int32
dtype: object
columns:
['country_id', 'product_id', 'year', 'M']
                 M
count 1,398,528.00
mean      0.148134
std       0.355233
min           0.00
25%           0.00
50%           0.00
75%           0.00
max       1.000000
Total memory usage: 12613228 bytes


Staging binary specialization matrix $M_{cp}$

In [38]:
# Save the binary specialization matrix a M_cp.parquet file
output_file_name = "M_cp.parquet"
save_hs92_parquet_data(
    data=M_cp, file_location=output_file_location, file_name=output_file_name
)

country_id column initial type:
category
country_id column final type:
int16
product_id column initial type:
category
product_id column final type:
int16
year column initial type:
category
year column final type:
int16


In [39]:
del RCA, M_cp
gc.collect()

0

Reading binary specialization matrix $M_{cp}$

In [40]:
input_file_name = "M_cp.parquet"
M_cp = read_hs92_parquet_data(output_file_location, input_file_name)

country_id column initial type:
int16
country_id column final type:
category
product_id column initial type:
int16
product_id column final type:
category
year column initial type:
int16
year column final type:
category


In [41]:
M_cp.head()

,country_id,product_id,year,M
0,8,650,2012,0
1,8,651,2012,0
2,8,652,2012,0
3,8,653,2012,0
4,8,654,2012,0


In [42]:
M_cp_unstacked = (
    M_cp.pivot(index=["country_id", "year"], columns="product_id", values="M")
    .fillna(0)
    .astype(int)
)
M_cp_unstacked.head(10)

product_id       650  651  652  653  654  655  656  657  658  659  ...  1883  \
country_id year                                                    ...         
8          2012    0    0    0    0    0    1    0    0    0    0  ...     0   
           2013    0    0    0    0    0    1    0    0    0    0  ...     0   
           2014    0    0    0    0    0    1    0    0    0    0  ...     0   
           2015    0    0    0    0    0    1    0    0    0    0  ...     0   
           2016    0    0    0    0    0    1    0    0    0    0  ...     0   
           2017    0    0    0    0    0    1    0    0    0    0  ...     0   
           2018    0    0    0    0    0    1    0    0    0    0  ...     0   
           2019    0    0    0    0    0    1    0    0    0    0  ...     0   
12         2012    0    0    0    0    0    0    0    0    0    0  ...     0   
           2013    0    0    0    0    0    0    0    0    0    0  ...     0   

product_id       1884  1885  1886  1887  1888  1889  1890  1891  1892  
country_id year                                                        
8          2012     0     0     0     0     1     1     0     0     1  
           2013     0     0     0     0     1     1     0     0     1  
           2014     0     0     0     0     0     0     0     1     1  
           2015     0     0     0     0     0     0     0     1     1  
           2016     0     0     0     0     0     0     0     1     0  
           2017     0     0     0     0     0     0     0     1     1  
           2018     0     0     0     0     0     0     0     1     0  
           2019     0     0     0     0     0     0     0     1     0  
12         2012     0     0     0     0     0     0     0     0     1  
           2013     0     0     0     0     0     0     0     0     1  

[10 rows x 1214 columns]

In [43]:
del M_cp_unstacked
gc.collect()

0

In [44]:
# assert False, "End of code"

### Calculating complexity indices ECI and PCI</br>
### staging the data into 'mor_implemented' catalog

In [45]:
order_of_reflections = 18

In [46]:
# Setting order of reflections. Was set in the beginning of the notebook
order = order_of_reflections

# Calculate n-order reflections
M_cp, ECI, PCI, K_c, K_p = n_order_reflections(M_cp, order)

['M_cp', <class 'pandas.core.frame.DataFrame'>]
shape:
(1395360, 80)
data types:
country_id    category
product_id    category
year          category
M                int32
k_c0             int32
                ...   
PCI_17         float64
k_c18          float64
k_p18          float64
ECI_18         float64
PCI_18         float64
Length: 80, dtype: object
columns:
['country_id', 'product_id', 'year', 'M', 'k_c0', 'k_p0', 'ECI_0', 'PCI_0', 'k_c1', 'k_p1', 'ECI_1', 'PCI_1', 'k_c2', 'k_p2', 'ECI_2', 'PCI_2', 'k_c3', 'k_p3', 'ECI_3', 'PCI_3', 'k_c4', 'k_p4', 'ECI_4', 'PCI_4', 'k_c5', 'k_p5', 'ECI_5', 'PCI_5', 'k_c6', 'k_p6', 'ECI_6', 'PCI_6', 'k_c7', 'k_p7', 'ECI_7', 'PCI_7', 'k_c8', 'k_p8', 'ECI_8', 'PCI_8', 'k_c9', 'k_p9', 'ECI_9', 'PCI_9', 'k_c10', 'k_p10', 'ECI_10', 'PCI_10', 'k_c11', 'k_p11', 'ECI_11', 'PCI_11', 'k_c12', 'k_p12', 'ECI_12', 'PCI_12', 'k_c13', 'k_p13', 'ECI_13', 'PCI_13', 'k_c14', 'k_p14', 'ECI_14', 'PCI_14', 'k_c15', 'k_p15', 'ECI_15', 'PCI_15', 'k_c16', 'k_p16', 'EC

Analyzing processed binary specialization matrix

In [47]:
df_stats(M_cp, "M_cp")

['M_cp', <class 'pandas.core.frame.DataFrame'>]
shape:
(1395360, 6)
data types:
year          category
country_id    category
product_id    category
M                int32
k_c0             int32
k_p0             int32
dtype: object
columns:
['year', 'country_id', 'product_id', 'M', 'k_c0', 'k_p0']
                 M         k_c0         k_p0
count 1,395,360.00 1,395,360.00 1,395,360.00
mean      0.148470       179.84        21.38
std       0.355565       125.97        10.75
min           0.00         7.00         2.00
25%           0.00        78.00        14.00
50%           0.00       158.00        19.00
75%           0.00       251.00        27.00
max       1.000000       560.00        80.00
Total memory usage: 68726960 bytes


In [48]:
M_cp.head(10)

,year,country_id,product_id,M,k_c0,k_p0
0,2012,8,650,0,166,16
1,2012,8,651,0,166,44
2,2012,8,652,0,166,18
3,2012,8,653,0,166,29
4,2012,8,654,0,166,32
5,2012,8,655,1,166,53
6,2012,8,656,0,166,26
7,2012,8,657,0,166,21
8,2012,8,658,0,166,16
9,2012,8,659,0,166,21


In [49]:
# assert False, "End of code"

Staging processed binary specialization matrix

In [50]:
# Saving binary country-product M_cp matrix to a .parquet file
output_file_name = "impl_M_cp.parquet"
save_hs92_parquet_data(
    data=M_cp, file_location=output_file_location, file_name=output_file_name
)

country_id column initial type:
category
country_id column final type:
int16
product_id column initial type:
category
product_id column final type:
int16
year column initial type:
category
year column final type:
int16


In [51]:
del M_cp
gc.collect()

0

Analyzing ECI 'reflections'

In [52]:
df_stats(ECI, "ECI")

['ECI', <class 'pandas.core.frame.DataFrame'>]
shape:
(1395360, 22)
data types:
year          category
country_id    category
k_c0             int32
ECI_0          float64
ECI_1          float64
ECI_2          float64
ECI_3          float64
ECI_4          float64
ECI_5          float64
ECI_6          float64
ECI_7          float64
ECI_8          float64
ECI_9          float64
ECI_10         float64
ECI_11         float64
ECI_12         float64
ECI_13         float64
ECI_14         float64
ECI_15         float64
ECI_16         float64
ECI_17         float64
ECI_18         float64
dtype: object
columns:
['year', 'country_id', 'k_c0', 'ECI_0', 'ECI_1', 'ECI_2', 'ECI_3', 'ECI_4', 'ECI_5', 'ECI_6', 'ECI_7', 'ECI_8', 'ECI_9', 'ECI_10', 'ECI_11', 'ECI_12', 'ECI_13', 'ECI_14', 'ECI_15', 'ECI_16', 'ECI_17', 'ECI_18']
              k_c0        ECI_0        ECI_1        ECI_2        ECI_3  \
count 1,395,360.00 1,395,360.00 1,395,360.00 1,395,360.00 1,395,360.00   
mean        179.84     0.000000 

In [53]:
ECI.head(10)

,year,country_id,k_c0,ECI_0,ECI_1,ECI_2,ECI_3,ECI_4,ECI_5,ECI_6,...,ECI_9,ECI_10,ECI_11,ECI_12,ECI_13,ECI_14,ECI_15,ECI_16,ECI_17,ECI_18
0,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
1,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
2,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
3,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
4,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
5,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
6,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
7,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
8,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829
9,2012,8,166,-0.131844,0.831169,-0.040967,0.548779,-0.090630,0.436914,-0.154422,...,0.349790,-0.239123,0.332050,-0.262405,0.321890,-0.277831,0.316103,-0.288045,0.312832,-0.294829


In [54]:
# Identifying duplicates
ECI.duplicated().sum()

1394208

Staging ECI 'reflections'

In [55]:
# Saving ECI (economic complexity) 'reflections' to a .parquet file
output_file_name = "impl_ECI.parquet"
save_hs92_parquet_data(
    data=ECI, file_location=output_file_location, file_name=output_file_name
)

country_id column initial type:
category
country_id column final type:
int16
year column initial type:
category
year column final type:
int16


In [56]:
del ECI
gc.collect()

0

Analyzing PCI 'reflections'

In [57]:
df_stats(PCI, "PCI")

['PCI', <class 'pandas.core.frame.DataFrame'>]
shape:
(1395360, 22)
data types:
year          category
product_id    category
k_p0             int32
PCI_0          float64
PCI_1          float64
PCI_2          float64
PCI_3          float64
PCI_4          float64
PCI_5          float64
PCI_6          float64
PCI_7          float64
PCI_8          float64
PCI_9          float64
PCI_10         float64
PCI_11         float64
PCI_12         float64
PCI_13         float64
PCI_14         float64
PCI_15         float64
PCI_16         float64
PCI_17         float64
PCI_18         float64
dtype: object
columns:
['year', 'product_id', 'k_p0', 'PCI_0', 'PCI_1', 'PCI_2', 'PCI_3', 'PCI_4', 'PCI_5', 'PCI_6', 'PCI_7', 'PCI_8', 'PCI_9', 'PCI_10', 'PCI_11', 'PCI_12', 'PCI_13', 'PCI_14', 'PCI_15', 'PCI_16', 'PCI_17', 'PCI_18']
              k_p0        PCI_0        PCI_1        PCI_2        PCI_3  \
count 1,395,360.00 1,395,360.00 1,395,360.00 1,395,360.00 1,395,360.00   
mean         21.38     0.000000 

In [58]:
PCI.head(10)

,year,product_id,k_p0,PCI_0,PCI_1,PCI_2,PCI_3,PCI_4,PCI_5,PCI_6,...,PCI_9,PCI_10,PCI_11,PCI_12,PCI_13,PCI_14,PCI_15,PCI_16,PCI_17,PCI_18
0,2012,650,16,-0.528296,0.041825,-0.163533,-0.095408,-0.047037,-0.064700,-0.051209,...,0.013322,-0.078547,0.038453,-0.086359,0.056339,-0.091186,0.068975,-0.094101,0.077868,-0.095848
1,2012,651,44,2.06,-0.925248,1.07,-0.692247,0.856319,-0.662381,0.761306,...,-0.664074,0.687405,-0.663612,0.673055,-0.662241,0.664986,-0.660650,0.660419,-0.659183,0.657825
2,2012,652,18,-0.343228,0.672375,-0.099111,0.783861,-0.358871,0.752922,-0.509731,...,0.709546,-0.635363,0.699732,-0.658968,0.694159,-0.671821,0.691076,-0.678889,0.689413,-0.682808
3,2012,653,29,0.674643,-1.72,1.74,-1.65,1.64,-1.62,1.59,...,-1.59,1.56,-1.58,1.55,-1.57,1.55,-1.57,1.55,-1.56,1.55
4,2012,654,32,0.952244,-0.131831,0.551835,0.144171,0.174042,0.171290,0.011671,...,0.152681,-0.096409,0.145214,-0.113888,0.140507,-0.122963,0.137682,-0.127799,0.136020,-0.130423
5,2012,655,53,2.90,-1.92,1.71,-1.94,1.61,-1.87,1.62,...,-1.77,1.65,-1.74,1.66,-1.72,1.67,-1.71,1.68,-1.70,1.68
6,2012,656,26,0.397041,-0.180307,0.379534,-0.156454,0.310714,-0.141430,0.241961,...,-0.135831,0.161820,-0.132818,0.142816,-0.129548,0.131554,-0.126501,0.124977,-0.123913,0.121167
7,2012,657,21,-0.065627,-1.85,0.963491,-1.29,1.12,-1.18,1.17,...,-1.16,1.17,-1.16,1.17,-1.16,1.16,-1.16,1.16,-1.16,1.16
8,2012,658,16,-0.528296,1.16,-0.764680,1.01,-0.815678,0.984618,-0.896043,...,0.996496,-0.984677,1.00,-1.00,1.01,-1.02,1.02,-1.02,1.02,-1.02
9,2012,659,21,-0.065627,-1.64,1.43,-1.58,1.51,-1.55,1.53,...,-1.53,1.52,-1.52,1.51,-1.52,1.50,-1.51,1.50,-1.51,1.50


Staging PCI 'reflections'

In [59]:
# Saving PCI (product complexity) 'reflections' to a .parquet file
output_file_name = "impl_PCI.parquet"
save_hs92_parquet_data(
    data=PCI, file_location=output_file_location, file_name=output_file_name
)

product_id column initial type:
category
product_id column final type:
int16
year column initial type:
category
year column final type:
int16


In [60]:
del PCI
gc.collect()

0

Analyzing diversity 'reflections'

In [61]:
df_stats(K_c, "K_c")

['K_c', <class 'pandas.core.frame.DataFrame'>]
shape:
(1395360, 21)
data types:
year          category
country_id    category
k_c0             int32
k_c1           float64
k_c2           float64
k_c3           float64
k_c4           float64
k_c5           float64
k_c6           float64
k_c7           float64
k_c8           float64
k_c9           float64
k_c10          float64
k_c11          float64
k_c12          float64
k_c13          float64
k_c14          float64
k_c15          float64
k_c16          float64
k_c17          float64
k_c18          float64
dtype: object
columns:
['year', 'country_id', 'k_c0', 'k_c1', 'k_c2', 'k_c3', 'k_c4', 'k_c5', 'k_c6', 'k_c7', 'k_c8', 'k_c9', 'k_c10', 'k_c11', 'k_c12', 'k_c13', 'k_c14', 'k_c15', 'k_c16', 'k_c17', 'k_c18']
              k_c0         k_c1         k_c2         k_c3         k_c4  \
count 1,395,360.00 1,395,360.00 1,395,360.00 1,395,360.00 1,395,360.00   
mean        179.84        28.75       248.93        27.30       263.31   
std     

In [62]:
K_c.head(10)

,year,country_id,k_c0,k_c1,k_c2,k_c3,k_c4,k_c5,k_c6,k_c7,...,k_c9,k_c10,k_c11,k_c12,k_c13,k_c14,k_c15,k_c16,k_c17,k_c18
0,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
1,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
2,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
3,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
4,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
5,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
6,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
7,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
8,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08
9,2012,8,166,32.31,253.34,28.09,268.64,27.33,272.57,27.15,...,27.10,273.97,27.09,274.05,27.09,274.07,27.09,274.08,27.09,274.08


Staging diversity 'reflections'

In [63]:
# Saving diversity 'reflections' to a .parquet file
output_file_name = "impl_K_c.parquet"
save_hs92_parquet_data(
    data=K_c, file_location=output_file_location, file_name=output_file_name
)

country_id column initial type:
category
country_id column final type:
int16
year column initial type:
category
year column final type:
int16


In [64]:
del K_c
gc.collect()

0

Analyzing ubiquity 'reflections'

In [65]:
df_stats(K_p, "K_p")

['K_p', <class 'pandas.core.frame.DataFrame'>]
shape:
(1395360, 21)
data types:
year          category
product_id    category
k_p0             int32
k_p1           float64
k_p2           float64
k_p3           float64
k_p4           float64
k_p5           float64
k_p6           float64
k_p7           float64
k_p8           float64
k_p9           float64
k_p10          float64
k_p11          float64
k_p12          float64
k_p13          float64
k_p14          float64
k_p15          float64
k_p16          float64
k_p17          float64
k_p18          float64
dtype: object
columns:
['year', 'product_id', 'k_p0', 'k_p1', 'k_p2', 'k_p3', 'k_p4', 'k_p5', 'k_p6', 'k_p7', 'k_p8', 'k_p9', 'k_p10', 'k_p11', 'k_p12', 'k_p13', 'k_p14', 'k_p15', 'k_p16', 'k_p17', 'k_p18']
              k_p0         k_p1         k_p2         k_p3         k_p4  \
count 1,395,360.00 1,395,360.00 1,395,360.00 1,395,360.00 1,395,360.00   
mean         21.38       281.31        26.10       271.38        26.63   
std     

In [66]:
K_p.head(10)

,year,product_id,k_p0,k_p1,k_p2,k_p3,k_p4,k_p5,k_p6,k_p7,...,k_p9,k_p10,k_p11,k_p12,k_p13,k_p14,k_p15,k_p16,k_p17,k_p18
0,2012,650,16,289.75,26.10,276.04,26.92,274.73,27.05,274.30,...,274.15,27.09,274.10,27.09,274.09,27.09,274.08,27.09,274.08,27.09
1,2012,651,44,234.41,28.55,267.68,27.37,272.52,27.15,273.67,...,273.97,27.09,274.05,27.09,274.08,27.09,274.08,27.09,274.08,27.09
2,2012,652,18,325.83,26.23,288.35,26.76,277.75,26.99,275.04,...,274.33,27.08,274.15,27.09,274.10,27.09,274.09,27.09,274.08,27.09
3,2012,653,29,188.86,29.91,254.32,27.76,268.97,27.26,272.74,...,273.73,27.10,273.99,27.09,274.06,27.09,274.08,27.09,274.08,27.09
4,2012,654,32,279.81,27.53,279.39,27.03,275.60,27.06,274.48,...,274.19,27.09,274.11,27.09,274.09,27.09,274.08,27.09,274.08,27.09
5,2012,655,53,177.32,29.84,250.17,27.75,268.05,27.26,272.54,...,273.69,27.10,273.98,27.09,274.06,27.09,274.08,27.09,274.08,27.09
6,2012,656,26,277.04,27.18,275.18,27.10,274.45,27.09,274.19,...,274.11,27.09,274.09,27.09,274.09,27.09,274.08,27.09,274.08,27.09
7,2012,657,21,181.29,28.35,259.30,27.50,270.61,27.21,273.18,...,273.84,27.10,274.02,27.09,274.07,27.09,274.08,27.09,274.08,27.09
8,2012,658,16,353.81,24.90,291.50,26.54,278.61,26.94,275.29,...,274.41,27.08,274.17,27.09,274.11,27.09,274.09,27.09,274.08,27.09
9,2012,659,21,193.24,29.28,255.27,27.70,269.23,27.25,272.81,...,273.75,27.10,273.99,27.09,274.06,27.09,274.08,27.09,274.08,27.09


Staging ubiquity 'reflections'

In [67]:
# Saving ubiquity 'reflections' to a .parquet file
output_file_name = "impl_K_p.parquet"
save_hs92_parquet_data(
    data=K_p, file_location=output_file_location, file_name=output_file_name
)

product_id column initial type:
category
product_id column final type:
int16
year column initial type:
category
year column final type:
int16


In [68]:
del K_p
gc.collect()

0

### Go to n05_MOR_complexity_cleaning.ipynb